---
### RNN/LSTM 코드 작성 시 주의사항

> 일단, torchtext의 BucketIterator는 배치별로 padding하여 배치내부에서 길이가 같게 만들어준다.그렇다면 rnn, lstm은 배치마다 길이가 달라도 되는가? RNN/LSTM의 경우 INPUT 길이가 다를 경우는 FOR문을 써줘야 한다. FOR문으로 반복적으로 사용하여 배치 내부 길이만큼 반복해줘야 한다. 하지만 배치별로 다른(그러니까 배치 내부에서 다른게 아니라면)거라면 for문 안써줘도된다. 그냥 알아서 인식한다. 대부분이 이렇게 구현되어있다. 

> StackOverflow 참고 :  Just use for-loop to iterate on you variant length sequences. But in the sense of efficiency, I would recommend you to use padded sequences for mini-batch. You can use the output of RNN to calculate loss and do backward.

> packed_Sequence 와 pad_packed_sequence 는 rnn이용하는거에서 필수다. 자세히보면, 배치사이즈내에서 pad토큰을 자동으로 무시하게끔 해주는게 pack_sequence 객체이다. 사용방법은 간단하고 pad_packed_sequence는 다시 원래 padding으로 돌려놓는것이다. packed_sequence할때 주의점은 embedding층 거친다음에 packed_pad_sequence해줘야한다. 아 물론 임베딩 층 코드보면 embed = nn.Embedding(vocab_size, embedding_size, padding_idx=0) 이렇게 해줘야 embedding층 거친 padding토큰 값은 모두 값이 0으로 나온다. 그러니 그다음에 pack_padded_sequence를 씌워주면 0들은 모두 padding 토큰으로 인식되는 것이다. 

>잘 보면 pack_padded_sequence 이다. 즉, padding은 일단 0으로 해준다음에 packing을 하는 것이다... 일단 공식문서 봐도 안나와있긴한데 pack_padded_sequence에서 default로 0을 pad token으로 인식하는듯하다. 음..이건 잘 모르겠다. 그냥 lengths를 무조건 지정해주면 알아서 인식을 하는건가...


> KERAS든 pYTORCH든 BATCH SIZE 간의 길이 차이는 문제가 없다. 그래서 우리가 layer에 입력으로 넣을때 timestamp를 인자로 넣어주지 않는것이다. 

> StackOverflow 참고 : This suggests that all the training examples have a fixed sequence length, namely timesteps.That is not quite correct, since that dimension can be None, i.e. variable length. Within a single batch, you must have the same number of timesteps )(this is typically where you see 0-padding and masking). But between batches there is no such restriction. During inference, you can have any length.)

> RNN에서 주의할점은 어떻게 RNN이 가변길이를 받을 수 있냐? 라고 생각할 수 있다. 그림으로 그려보면 이게 말이 안되거든. 근데 이건 RECURRENT한 것이다. 즉, UNFOLD해서 보면 가변길이가 불가능해보이지만 실상은 하나가 덮어씌워지는 것이기 때문에 가능한 것이다 ! 그렇게 이해해야 한다. 

>The number of cells and sequence length are distinct concepts. 다르게 적용가능하다. cell 개수와 timestep 개수는 완전히 별개라고 생각해야한다. 전혀 상관없이 독립적으로 움직임. 

> 아래가 가장 완벽한 설명. timestamp의 길이는 아무 상관이 없다. 그림으로 외운것처럼 10개의 timestamp cell이 10개여야하는것같지만 그게 아니고 cell 개수랑 관련없이 rnn은 한개의 cell에 다시 넣고넣고 하는것이다. 즉, timestamp길이가 아무리 달라도 상관없다. 그래서keras, pytorch에서 sequence length를 지정하지 않는것이다. 그렇다면 역전파시에는? 아마, for range(timestamp)하면서 w, b값들은 저장을 해놓을 것이다. 왜냐하면 앞에 w이용해서 계산한게 뒤에 w가 되기때문에, 저장도 해놨을 것. 

>StackOverflow 참고 : The point of the RNN is that, in your terminology, w1...w10 are actually all the same sets of weights. The weights are not different for different time steps. So really you have w1=w2=w3=...=w10=w. There are only one set of weights for an RNN regardless of how long the input sequence is. Conceptually you are feeding your outputs back into the inputs of the same neuron - hence recurrence. You therefore only need to store one W which is reused at every time step.

>그렇다면 배치별로는 왜 같아야 하나? 이건 행렬연산때문에 그런 것이다. rnn과 상관이 있는게 아니다. 
> pack_padded_sequence는 배치내에서 계산량을 줄이기 위해 있는것이다. 

> 이렇기에 torch에서 seq2seq 구현할때 for문을 쓸 수 있는것이다. for문으로 원하는 길이만큼만 뽑아낼 수 있고 이건 원래 rnn방식과 동일한것이다. hidden state를 그대로 다시 넣어주므로!!!!!



In [16]:
input = torch.tensor([[  1,  16,   7,  11,  13,   2],
        [  1,  16,   6,  15,   8,   0],
        [  5,  14,   3,  17,   0,   0],
        [ 12,   9,   0,   0,   0,   0],
        [ 10,   0,   0,   0,   0,   0]])

In [11]:
import torch

## 이런식으로 하는 것. 0이 default인 건 아닌듯하다고 length는 지정을 해줘야하는듯. 

pack = torch.nn.utils.rnn.pack_padded_sequence(torch.tensor([[  1,  16,   7,  11,  13,   2],
        [  1,  16,   6,  15,   8,   0],
        [  5,  14,   3,  17,   0,   0],
        [ 12,   9,   0,   0,   0,   0],
        [ 10,   0,   0,   0,   0,   0]]), lengths=[6,5,4,2,1], batch_first=True)

In [14]:
print(pack[0])
print(pack[1])

tensor([ 1,  1,  5, 12, 10, 16, 16, 14,  9,  7,  6,  3, 11, 15, 17, 13,  8,  2])
tensor([5, 4, 3, 3, 2, 1])


In [17]:
embed = torch.nn.Embedding(20, 4, padding_idx=0)
gru = torch.nn.RNN(input_size=4, hidden_size=3, num_layers=1, bidirectional=False, batch_first=True)

embeded = embed(input)

In [18]:
embeded ## 잘 보면 0으로 채워져있다 padding_idx에 대해서는 

tensor([[[-0.6356,  0.0299,  1.1349, -0.7978],
         [-0.1216,  2.7567, -0.8621, -0.3365],
         [-1.9938, -0.1446, -1.6731, -0.0235],
         [-1.8181, -0.3500, -1.6957,  0.2605],
         [-1.1674,  1.2159, -2.1042, -0.8860],
         [-0.3848, -0.8167,  1.4815, -0.3622]],

        [[-0.6356,  0.0299,  1.1349, -0.7978],
         [-0.1216,  2.7567, -0.8621, -0.3365],
         [-0.5168,  0.5046, -0.0534, -1.5177],
         [-0.3553, -0.2955, -0.1902, -0.4916],
         [ 0.0485, -1.7518, -0.1442, -0.3059],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 2.0881,  1.0330,  0.9342,  0.2522],
         [-0.1899,  0.8890, -0.6595,  0.7455],
         [ 0.0727, -0.6514, -1.2202,  0.4421],
         [-1.1495, -0.8171,  0.0947,  1.0072],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[-0.1983,  0.0173, -0.6123, -0.0224],
         [ 1.1602, -0.5361,  0.7760,  0.5164],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
       

In [20]:
packed_input =  torch.nn.utils.rnn.pack_padded_sequence(embeded, lengths=[6,5,4,2,1], batch_first=True)
packed_output, hidden = gru(packed_input)

In [28]:
packed_output ## gru를 통과했는데도 packed 객체가 나온다. 그래서 pad_packed_sequence를 쓸 수 있다
## 그리고 주의점이 packed_output에는 데이터별로 구분되어있는게 아니고 한줄로 나온다. 
## 그래서 pad_packed_sequence를 통해서 바꿔줘야하는것이다. 

PackedSequence(data=tensor([[ 0.5226,  0.1395,  0.3603],
        [ 0.5226,  0.1395,  0.3603],
        [ 0.2775,  0.7649,  0.8869],
        [ 0.6255,  0.2829, -0.0848],
        [ 0.7768,  0.4082,  0.1373],
        [ 0.9831,  0.2901, -0.9050],
        [ 0.9831,  0.2901, -0.9050],
        [ 0.7326,  0.1053, -0.2151],
        [-0.3962,  0.5280,  0.8547],
        [ 0.9044, -0.0951, -0.9513],
        [ 0.9574, -0.0822, -0.6585],
        [ 0.3420,  0.2655, -0.2556],
        [ 0.7812,  0.1362, -0.9418],
        [ 0.6180,  0.1987, -0.4311],
        [-0.2773,  0.3356,  0.0980],
        [ 0.9912, -0.0121, -0.9754],
        [-0.0961,  0.0521,  0.5715],
        [-0.1416,  0.3638,  0.4573]], grad_fn=<CatBackward0>), batch_sizes=tensor([5, 4, 3, 3, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [34]:
torch.nn.utils.rnn.pad_packed_sequence(packed_output, batch_first = True)[0]
## 위에거가 아래로 변환되는건 약간 머리를 써야한다. 그러니 너무 걱정말고 그냥 pad_packed_sequence 쓰면된다.
### batch_first = True 꼭 해줘야한다. 안해주면 망함. 

tensor([[[ 0.5226,  0.1395,  0.3603],
         [ 0.9831,  0.2901, -0.9050],
         [ 0.9044, -0.0951, -0.9513],
         [ 0.7812,  0.1362, -0.9418],
         [ 0.9912, -0.0121, -0.9754],
         [-0.1416,  0.3638,  0.4573]],

        [[ 0.5226,  0.1395,  0.3603],
         [ 0.9831,  0.2901, -0.9050],
         [ 0.9574, -0.0822, -0.6585],
         [ 0.6180,  0.1987, -0.4311],
         [-0.0961,  0.0521,  0.5715],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.2775,  0.7649,  0.8869],
         [ 0.7326,  0.1053, -0.2151],
         [ 0.3420,  0.2655, -0.2556],
         [-0.2773,  0.3356,  0.0980],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.6255,  0.2829, -0.0848],
         [-0.3962,  0.5280,  0.8547],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.7768,  0.4082,  0.1373],
         [ 0.0000,  0.0000,  0.0000],
    

In [ ]:
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
# tensor([[ 1,  2,  3],
#    [ 3,  4,  0]])

torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])

#PackedSequence(data=tensor([ 1,  3,  2,  4,  3]), batch_sizes=tensor([ 2,  2,  1]))

In [35]:
test_layer = torch.nn.RNN(input_size=4, hidden_size=3, num_layers=1, bidirectional=False, batch_first=True)

In [42]:
print(test_layer(torch.FloatTensor([[[1,2,3,5],[1,2,3,4]]])))  ## 같은 layer에서 위는 timestemp 2 밑은 timestamp 3인경우다. 배치안에서만 길이가 같으면되지, 
### 배치끼리는 상관없는것의 증명이다. 이건 NLP의 기본. RNN 자체가 그렇게 설계되어있는듯하다. 
### 아니면 가변해서 CELL을 추가하는건가? 그렇게 생각하면 나을것같다. 
print(test_layer((torch.FloatTensor([[[1,2,3,5],[1,2,3,4], [100,101,102,103]]]))))

(tensor([[[-0.9868, -0.8875, -0.9981],
         [-0.9978, -0.9717, -0.9977]]], grad_fn=<TransposeBackward1>), tensor([[[-0.9978, -0.9717, -0.9977]]], grad_fn=<StackBackward0>))
(tensor([[[-0.9868, -0.8875, -0.9981],
         [-0.9978, -0.9717, -0.9977],
         [-1.0000, -1.0000, -1.0000]]], grad_fn=<TransposeBackward1>), tensor([[[-1., -1., -1.]]], grad_fn=<StackBackward0>))


In [43]:
print(test_layer(torch.FloatTensor([[[1,2,3,5],[1,2,3]]])))  ### 일단, BATCH 내에서 길이가 다른건 애초에 TENSOR에서 용납이 안된다. 

ValueError: expected sequence of length 4 at dim 2 (got 3)

> Pytorch Crossentropyloss의 경우는 label 원핫인코딩 안해도된다. 그냥 숫자 한개만 던져줘도 됨

In [45]:
embed_layer = torch.nn.Embedding(20,4) ## 그냥 참고용으로 vocab_size = 20으로 하면 숫자 0~19까지 default로 할당되는듯. 

In [52]:
print(embed_layer(torch.tensor([0])))  ## embedding layer는 timestamp길이가 전혀 상관이 없다. 몇개가 들어오든 그 개수만큼 출력해줄 뿐. 
print(embed_layer(torch.tensor([0,1])))

tensor([[ 0.8720, -0.5460, -1.8625,  0.8421]], grad_fn=<EmbeddingBackward0>)
tensor([[ 0.8720, -0.5460, -1.8625,  0.8421],
        [-1.6213,  0.7283, -0.5763,  1.3026]], grad_fn=<EmbeddingBackward0>)


In [ ]:
## 예시코드로서 배치내에서 길이가 다를경우에는 for문을 써줘야한다. 
class DeepIO(nn.Module):
    def __init__(self):
        super(DeepIO, self).__init__()
        self.rnn = nn.LSTM(input_size=6, hidden_size=512,
                           num_layers=2, bidirectional=True)
        self.drop_out = nn.Dropout(0.25)
        self.fc1 = nn.Linear(512, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc_out = nn.Linear(256, 7)

    def forward(self, x):
        """
        args:
        x:  a list of inputs of diemension [BxTx6]
        """
        outputs = []
        # iterate in the batch through all sequences
        for xx in x:
            s, n = xx.shape
            out, hiden = self.rnn(xx.unsqueeze(1))
            out = out.view(s, 1, 2, 512)
            out = out[-1, :, 0]
            outputs.append(out.squeeze())
        outputs = torch.stack(outputs)

        y = F.relu(self.fc1(outputs), inplace=True)
        y = self.bn1(y)
        y = self.drop_out(y)
        y = self.out(y)
        return y

---
### Seq2Seq

In [ ]:
## 아래코드같은 경우 SEQ2SEQ에서 encoder 부분에 sos 토큰을 넣어주었다. 

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.xhid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src = [src len, batch_size]
        embedded = self.dropout(self.embedding(src))
        
        # embedded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        # Decoder에서 항상 n directions = 1
        # 따라서 hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]
        
        # input = [1, batch size]
        input = input.unsqueeze(0)
        
        # embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        # Decoder에서 항상 seq len = n directions = 1 
        # 한 번에 한 토큰씩만 디코딩하므로 seq len = 1
        # 따라서 output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        
        # prediction = [batch size, output dim]
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
   def __init__(self, encoder, decoder, device):
       super().__init__()
       
       self.encoder = encoder
       self.decoder = decoder
       self.device = device
       
       # Encoder와 Decoder의 hidden dim이 같아야 함
       assert encoder.hid_dim == decoder.hid_dim
       # Encoder와 Decoder의 layer 개수가 같아야 함
       assert encoder.n_layers == decoder.n_layers
       
   def forward(self, src, trg, teacher_forcing_ratio=0.5):
       # src = [src len, batch size]
       # trg = [trg len, batch size]
       
       trg_len = trg.shape[0]
       batch_size = trg.shape[1]
       trg_vocab_size = self.decoder.ouput_dim
       
       # decoder 결과를 저장할 텐서
       outputs = torch.zeros(trg_len, batch_size, trg_vocab_size)
       
       # Encoder의 마지막 은닉 상태가 Decoder의 초기 은닉상태로 쓰임
       hidden, cell = self.encoder(src)
       
       # Decoder에 들어갈 첫 input은 <sos> 토큰
       input = trg[0, :]
       
       # target length만큼 반복
       # range(0,trg_len)이 아니라 range(1,trg_len)인 이유 : 0번째 trg는 항상 <sos>라서 그에 대한 output도 항상 0 
       for t in range(1, trg_len):
           output, hidden, cell = self.decoder(input, hidden, cell)
           outputs[t] = output
           
           # random.random() : [0,1] 사이 랜덤한 숫자 
           # 랜덤 숫자가 teacher_forcing_ratio보다 작으면 True니까 teacher_force=1
           teacher_force = random.random() < teacher_forcing_ratio
           
           # 확률 가장 높게 예측한 토큰
           top1 = output.argmax(1) 
           
           # techer_force = 1 = True이면 trg[t]를 아니면 top1을 input으로 사용
           input = trg[t] if teacher_force else top1
       
       return outputs

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uiform_(param.data, -0.08, 0.08)

model.apply(init_weights)

optimizer = optim.Adam(model.parameters())

### 이것도 중요한게 encoder input의 입력을 ignore하는게 아니고 target의 padding 토큰을 ignore하는것. 

### ignore_index : 특정 Class에 대하여 Loss 계산을 제외하겠다는 인자이다. Detection이나 Segmentation에서 사용자가 특정 Class를 제외하고 싶을때 사용하면 편하다.
### 원래대로라면 Model의 마지막 Layer인 FC Layer부분의 채널을 변경하고 원-핫 인코딩을 다시 해주어야한다. 하지만 해당 인자를 사용하면 이러한 번거로움을 줄일 수 있다.

# <pad> 토큰의 index를 넘겨 받으면 오차 계산하지 않고 ignore하기
# <pad> = padding
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = trg_pad_idx)

In [ ]:
##잘 보면 loss를 output 전체와 label 전체와 한번에 비교해서 계산하였음. 

#“Teacher forcing”은 다음 입력으로 디코더의 예측을 사용하는 대신 실제 목표 출력을 다음 입력으로 사용하는 컨셉입니다. “Teacher forcing”을 사용하면 수렴이 빨리되지만 학습된 네트워크가 잘못 사용될 때 불안정성을 보입니다..
#Teacher-forced 네트워크의 출력이 일관된 문법으로 읽지만 정확한 번역과는 거리가 멀다는 것을 볼 수 있습니다. 직관적으로 출력 문법을 표현하는 법을 배우고 교사가 처음 몇 단어를 말하면 의미를 “선택” 할 수 있지만, 번역에서 처음으로 문장을 만드는 법은 잘 배우지 못합니다.
#PyTorch의 autograd 가 제공하는 자유 덕분에 간단한 If 문으로 Teacher Forcing을 사용할지 아니면 사용하지 않을지를 선택할 수 있습니다. 더 많이 사용하려면 teacher_forcing_ratio 를 확인하십시오.


# 사실상 다음과 같이 해야 맞다. 이 경우에는 target_length를 쓸 수 있다. 예측이 아니고 학습과정에서 teacher foroing을 쓸지말지 결정하므로
## 그래서 랜덤하게 비율을 선택해서 어느때는 teacher forcing하고 어는때는 하지 않고 이런것..

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break




def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss=0
    
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        
        # loss 함수는 2d input으로만 계산 가능 
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # trg = [(trg len-1) * batch size]
        # output = [(trg len-1) * batch size, output dim)]
        loss = criterion(output, trg)
        
        loss.backward()
        
        # 기울기 폭발 막기 위해 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss+=loss.item()
        
    return epoch_loss/len(iterator)

In [ ]:
## 사실상, 예측시에는 target이 어떤 길이에서 끝날지 모르기 때문에 이 코드는 잘못되었다고 생각
## 아래 셀 코드가 정확하다.

def evaluate(model, iterator, criterion):
    model.eval()
    eopch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            
            # teacher_forcing_ratio = 0 (아무것도 알려주면 안 됨)
            output = model(src, trg, 0)
            
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]
            
            loss = criterion(output, trg)
            
            epoch_loss+=loss.item()
        
        return epoch_loss/len(iterator)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')